In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import pickle
import joblib
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

class CandleFit:
    def __init__(self, ticker: str, period: str = '5y'):
        """
        Initializes the CandleFit object with a specified ticker and period.

        Parameters:
        ticker (str): The ticker symbol of the stock.
        period (str): The period for which to download historical data.
        """
        self.ticker = ticker
        self.period = period
        self.data = self.get_ticker()

    def get_ticker(self):
        """
        Downloads historical data for the specified ticker and period.

        Returns:
        pd.DataFrame: A DataFrame containing the historical data with formatted dates and column names.
        """
        try:
            ticker_obj = yf.Ticker(self.ticker)
            hist = ticker_obj.history(period=self.period)
            hist.index = pd.to_datetime(hist.index).strftime('%Y-%m-%d')
            hist.columns = [col.lower() for col in hist.columns]
            return hist
        except Exception as e:
            print(f"Error downloading aux: {e}")
            return pd.DataFrame()
    
    @staticmethod
    def load_dict(key: str) -> dict:
        """
        Loads a dictionary from a pickle file based on the provided key.

        Parameters:
        key (str): The key to search for in the dictionary.

        Returns:
        dict: The dictionary associated with the provided key.

        Raises:
        FileNotFoundError: If the pickle file is not found.
        KeyError: If the key is not found in the dictionary.
        ValueError: If there is an error loading the pickle file.
        """
        filepath = os.path.join('..', 'pkl', 'threshold_dicts.pkl')
        try:
            with open(filepath, 'rb') as file:
                data = pickle.load(file)
            
            for item in data:
                if key in item:
                    return item[key]
            raise KeyError(f"Key '{key}' not found in the threshold dictionary.")
        except FileNotFoundError:
            raise FileNotFoundError(f"The file '{filepath}' was not found.")
        except pickle.PickleError:
            raise ValueError("Error occurred while loading the pickle file.")
        
    def get_price_features(self):
        """
        Calculates various price and volume features from historical data.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated features.
        """
        aux = self.data.copy()
        df = pd.DataFrame(index=aux.index)
        df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
        df['return_rate'] = aux['close'].pct_change()
        df['volume_change'] = aux['volume'].diff()
        df['volume_var'] = aux['volume'].pct_change() + 1
        df['price_range'] = aux['high'] - aux['low']
        df['price_var'] = df['price_range'] / aux['low']
        df['price_change'] = aux['close'] - aux['open']
        df['close_vol'] = aux['close'].expanding().std()
        df['low_vol'] = aux['low'].expanding().std()
        df['high_vol'] = aux['high'].expanding().std()
        df['open_vol'] = aux['open'].expanding().std()
        df['upper_wick'] = aux['high'] - aux[['open', 'close']].max(axis=1)
        df['lower_wick'] = aux[['open', 'close']].min(axis=1) - aux['low']
        df['wick_change'] = df['upper_wick'] - df['lower_wick']
        df['wick_var'] = df['wick_change'] / df['lower_wick']
        df['wick_vol'] = df['wick_change'].abs().expanding().std()
        df = df.apply(pd.to_numeric, errors='coerce')
        df = pd.concat([aux, df], axis=1)
        
        return df

    def get_candle_features(self, threshold_dict: dict = None):
        """
        Calculates various candlestick features based on price data and a threshold dictionary.

        Parameters:
        threshold_dict (dict): A dictionary containing thresholds for calculating features. If None, it loads the default dictionary.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated candlestick features.
        """
        aux = self.get_price_features()
        if threshold_dict is None:
            threshold_dict = self.load_dict(key='threshold_dict')
        
        df = pd.DataFrame(index=aux.index)
        
        df[f'std_{threshold_dict["volatility_window"]}'] = aux['price_change'].rolling(window=threshold_dict['volatility_window']).std().abs()
        df['bearish_threshold'] = 1 + threshold_dict["bearish_threshold"] * df[f'std_{threshold_dict["volatility_window"]}'] 
        df['bullish_threshold'] = 1 + threshold_dict["bullish_threshold"] * df[f'std_{threshold_dict["volatility_window"]}'] 
        df['is_bearish'] = (aux['close'] <= (aux['open'] - df['bearish_threshold'])).astype(int)
        df['is_bullish'] = (aux['close'] >= (aux['open'] + df['bullish_threshold'])).astype(int)
        df['is_doji'] = (abs(aux['close'] - aux['open']) <= threshold_dict['doji_threshold']).astype(int)
        df['is_bearish_open_gap'] = (aux['close'] < aux['open'].shift(1)).astype(int)
        df['is_bullish_open_gap'] = (aux['open'] > aux['close'].shift(1)).astype(int)
        
        df = pd.concat([aux, df], axis=1)

        return df

    def fit_morning_star(self, threshold_dict: dict = None):
        """
        Identifies the morning star candlestick pattern in the historical data.

        Parameters:
        threshold_dict (dict): A dictionary containing thresholds for the morning star pattern. If None, it loads the default dictionary.

        Returns:
        pd.DataFrame: A DataFrame with a column indicating the presence of the morning star pattern.
        """
        df = self.get_candle_features()  
        if threshold_dict is None:
            threshold_dict = self.load_dict(key='morning_star_dict')
            display(pd.DataFrame([threshold_dict]))


        # Condition 1: Two days ago was a bearish candle and the close of that day is lower than the open of that day adjusted by the threshold
        condition1 = (df['is_bearish'].shift(2) == 1) & \
                    (df['close'].shift(2) + threshold_dict['bearish_threshold'] * df['price_change'].shift(2) <= df['open'].shift(2))

        # Condition 2: The previous day was a doji candle and had a bearish open gap
        condition2 = (df['is_bearish_open_gap'].shift(1) == 1) & (df['is_doji'].shift(1) == 1)

        # Condition 3: Today is a bullish candle and the price change from two days ago to today is significant
        condition3 = (df['is_bullish'] == 1) & \
                    (df['close'] - df['close'].shift(2) >= threshold_dict['bullish_threshold'] * df['price_change'].shift(2).abs())

        # Combine all conditions to determine the morning star pattern
        df['is_morning_star'] = ((condition1) & (condition2) & (condition3)).astype(int)
        
        return df
    
    def get_movings(self, 
                threshold_dict: dict = None,
                short_window=None,
                long_window=None,
                signal_window=None):
        """
        Calculates buy and sell signals based on moving averages for different strategies and parameters.

        Parameters:
        - threshold_dict (dict, optional): Dictionary containing moving average settings for different strategies. If None, it will be loaded with `self.load_dict(key='rolling_cross_dict')`.
        - short_window (int, optional): Time window for the short moving average. Not used directly in the function.
        - long_window (int, optional): Time window for the long moving average. Not used directly in the function.
        - signal_window (int, optional): Time window for signal calculation. Not used directly in the function.

        Returns:
        - pd.DataFrame: DataFrame with additional columns for moving averages and buy/sell signals.
        """
    
        df = self.get_candle_features()  
        if threshold_dict is None:
            threshold_dict = self.load_dict(key='rolling_cross_dict')
            display(pd.DataFrame([threshold_dict]))
            
        for key, value in threshold_dict.items():
            strategy = key.split('_')[0]
            short = value['short']
            long = value['long']
            
            df[f'{strategy}_short_{short}'] = df['close'].rolling(window=short, min_periods=1).mean()
            df[f'{strategy}_long_{long}'] = df['close'].rolling(window=long, min_periods=1).mean()
            
            buy = (df[f'{strategy}_short_{short}'] > df[f'{strategy}_long_{long}']) & \
                  (df[f'{strategy}_short_{short}'] < df[f'{strategy}_long_{long}'].shift(1))
            
            sell = (df[f'{strategy}_short_{short}'] < df[f'{strategy}_long_{long}']) &  \
                (df[f'{strategy}_short_{short}'] > df[f'{strategy}_long_{long}'].shift(1))
            
            df[f'{strategy}_{short}_{long}'] = np.where(buy, 1, np.where(sell, -1, 0))
                
        return df           

    def candlestick_chart(self, 
                      key: str = 'is_morning_star', 
                      plot_type: str = 'pattern',
                      height: int = 900, 
                      offset: float = 20):
        """
        Generates a candlestick chart with optional pattern or price action markers.

        Parameters:
        - key (str): Key for identifying the pattern or price action indicators. Defaults to 'is_morning_star'.
        - plot_type (str): Type of plot to generate. Options are 'pattern' or 'price_action'. Defaults to 'pattern'.
        - height (int): Height of the plot in pixels. Defaults to 900.
        - offset (float): Vertical offset for pattern markers. Defaults to 20.

        Returns:
        - go.Figure: A Plotly Figure object with the candlestick chart.
        """
    
        if plot_type == 'pattern': 
            if key == 'morning_star':
                aux = self.fit_morning_star()
            else:
                aux=self.get_candle_features()
            
            if key not in aux.columns:
                print(f"Key '{key}' not found in aux columns.")
                return None
                
            
            markers = aux[aux[key] == 1].copy()
            markers['close'] = markers['low'] - offset
            markers = markers.dropna(subset=['close'])
            marker_trace = go.Scatter(
                x=markers.index,
                y=markers['close'],
                mode='markers',
                marker=dict(
                    color='blue',
                    size=8,
                    symbol='triangle-up'
                ),
                name=key,
                yaxis="y"
            )
        
        elif plot_type == 'price_action':
            aux = self.get_movings()
            strategy, short, long = key.split('_')
            short_col = f'{strategy}_short_{short}'
            long_col = f'{strategy}_long_{long}'
            signal_col = f'{strategy}_{short}_{long}'
            
            # Plotando as linhas das médias móveis
            short_trace = go.Scatter(
                x=aux.index,
                y=aux[short_col],
                mode='lines',
                name=f'{strategy.capitalize()} Short {short}',
                line=dict(color='purple')  # Cor roxa para a média curta
            )
            long_trace = go.Scatter(
                x=aux.index,
                y=aux[long_col],
                mode='lines',
                name=f'{strategy.capitalize()} Long {long}',
                line=dict(color='orange')  # Cor laranja para a média longa
            )

            # Processamento dos marcadores
            markers = aux[aux[signal_col] != 0].copy()  # Filtra apenas onde o sinal é -1 ou 1
            markers['close'] = markers.apply(
                lambda row: row['low'] - offset if row[signal_col] == 1 else row['high'] + offset,
                axis=1
            )
            markers['color'] = markers[signal_col].apply(lambda x: 'green' if x == 1 else 'red')
            markers['symbol'] = markers[signal_col].apply(lambda x: 'arrow-up' if x == 1 else 'arrow-down')
            
            marker_trace = go.Scatter(
                x=markers.index,
                y=markers['close'],
                mode='markers',
                marker=dict(
                    color=markers['color'],
                    size=8,
                    symbol=markers['symbol']
                ),
                name='Signal',
                yaxis="y"
            )
            
            trace = go.Candlestick(
                x=aux.index,
                open=aux["open"],
                high=aux["high"],
                low=aux["low"],
                close=aux["close"],
                name=self.ticker,
                yaxis="y"
            )
            
            # Cores para o volume
            volume_colors = ['green' if aux['volume'][i] > aux['volume'][i-1] else 'red' for i in range(1, len(aux))]
            volume_colors.insert(0, 'green')
            volume_trace = go.Bar(
                x=aux.index,
                y=aux['volume'],
                marker_color=volume_colors,
                name='Volume',
                yaxis="y2"
            )
            
            layout = go.Layout(
                title=f"{self.ticker} Candlestick Chart markers: {key.capitalize()}",
                xaxis=dict(title="Date"),
                yaxis=dict(title="Price", domain=[0.3, 1]),
                yaxis2=dict(title="Volume", domain=[0, 0.2]),
                height=height,
                barmode='relative'
            )
            
            fig = go.Figure(data=[trace, short_trace, long_trace, marker_trace, volume_trace], layout=layout)
            return fig

In [4]:

ticker = CandleFit('AAPL')
ticker.candlestick_chart(key='sma_10_30', plot_type='price_action')

,sma_5_l0,sma_5_20,sma_10_30,sma_10_50,sma_20_50,sma_20_100,sma_50_100,sma_50_150,sma_100_200,ema_5_10,ema_5_20,ema_10_30,ema_10_50,ema_20_50,ema_20_100,ema_50_100,ema_50_150,ema_100_200,macd_9_12,macd_9_30,macd_12_26,macd_12_50,macd_26_50,macd_26_100,macd_50_100,macd_50_200
0,"{'short': 5, 'long': 10}","{'short': 5, 'long': 20}","{'short': 10, 'long': 30}","{'short': 10, 'long': 50}","{'short': 20, 'long': 50}","{'short': 20, 'long': 100}","{'short': 50, 'long': 100}","{'short': 50, 'long': 150}","{'short': 100, 'long': 200}","{'short': 5, 'long': 10}","{'short': 5, 'long': 20}","{'short': 10, 'long': 30}","{'short': 10, 'long': 50}","{'short': 20, 'long': 50}","{'short': 20, 'long': 100}","{'short': 50, 'long': 100}","{'short': 50, 'long': 150}","{'short': 100, 'long': 200}","{'short': 12, 'long': 26}","{'short': 30, 'long': 60}","{'short': 12, 'long': 26}","{'short': 12, 'long': 50}","{'short': 26, 'long': 50}","{'short': 26, 'long': 100}","{'short': 50, 'long': 100}","{'short': 50, 'long': 200}"


In [ ]:
ticker.get_movings().columns